In [1]:
import numpy as np
import os
import mne
from scipy.signal import stft
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
def load_bci_iv_2b(subject=1, data_dir="./BCI_IV_2b"):
    """Load EEG data for a subject from BCI Competition IV 2b."""
    # Construct file path (e.g., "./BCI_IV_2b/B01T.gdf")
    file_path = os.path.join(data_dir, f"B0{subject}T.gdf")
    
    # Load the GDF file (correct function for .gdf)
    raw = mne.io.read_raw_gdf(file_path, preload=True)
    
    # Extract events (annotations)
    events, event_dict = mne.events_from_annotations(raw)
    
    # Extract left/right hand MI trials (labels: 1=left, 2=right)
    epochs = mne.Epochs(raw, events, event_id={'left': 1, 'right': 2}, 
                       tmin=0.5, tmax=2.5, baseline=None, preload=True)
    
    X = epochs.get_data()  # Shape: (n_trials, 3 channels, 500 timepoints)
    y = epochs.events[:, 2] - 1  # Convert labels to 0/1
    
    return X, y

In [10]:
data_dir = "./BCICIV_2b_gdf"  # Update this path!
X, y = load_bci_iv_2b(subject=101, data_dir=data_dir)

print(f"Data shape: {X.shape}")  # Should print (n_trials, 3, 500)
print(f"Labels: {np.unique(y)}")  # Should print [0 1] (left/right)

Extracting EDF parameters from c:\Users\luize\OneDrive\Área de Trabalho\Codes\EEG_Study\BCICIV_2b_gdf\B0101T.gdf...
GDF file detected


OverflowError: Python integer 256 out of bounds for uint8

In [4]:
def compute_stft(eeg_data, fs=250, nperseg=64, noverlap=50):
    """Convert EEG trials to STFT spectrograms."""
    n_trials, n_channels, _ = eeg_data.shape
    stft_images = []
    
    for trial in range(n_trials):
        channel_images = []
        for channel in range(n_channels):
            f, t, Zxx = stft(eeg_data[trial, channel], fs=fs, nperseg=nperseg, noverlap=noverlap)
            # Extract mu (6-13 Hz) and beta (17-30 Hz) bands
            mu_band = Zxx[(f >= 6) & (f <= 13), :]
            beta_band = Zxx[(f >= 17) & (f <= 30), :]
            beta_band = np.resize(beta_band, (15, 32))  # Resize to match mu_band
            combined = np.vstack((mu_band.real, beta_band.real))  # Shape: (31, 32)
            channel_images.append(combined)
        
        # Stack channels (3 electrodes × 31×32 → 93×32)
        stft_image = np.vstack(channel_images)
        stft_images.append(stft_image)
    
    return np.array(stft_images)

# Apply STFT
X_stft = compute_stft(X)
print(f"STFT shape: {X_stft.shape}")  # (n_trials, 93, 32)

NameError: name 'X' is not defined

In [ ]:
def build_cnn(input_shape=(93, 32, 1)):
    """CNN for feature extraction."""
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(30, kernel_size=(93, 3), activation='relu', padding='valid'),
        layers.MaxPooling2D(pool_size=(1, 10)),
        layers.Flatten()
    ])
    return model

def build_vae(latent_dim=10):
    """Variational Autoencoder for classification."""
    # Encoder
    encoder_inputs = layers.Input(shape=(900,))
    x = layers.Dense(512, activation='relu')(encoder_inputs)
    x = layers.Dense(256, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    
    # Sampling layer
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    z = layers.Lambda(sampling)([z_mean, z_log_var])
    
    # Decoder
    decoder_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(256, activation='relu')(decoder_inputs)
    x = layers.Dense(512, activation='relu')(x)
    decoder_outputs = layers.Dense(900, activation='sigmoid')(x)
    
    # VAE models
    encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z])
    decoder = models.Model(decoder_inputs, decoder_outputs)
    
    # Classification head
    classifier_outputs = layers.Dense(1, activation='sigmoid')(z)
    vae_classifier = models.Model(encoder_inputs, classifier_outputs)
    
    return encoder, decoder, vae_classifier

# Combine CNN and VAE
cnn = build_cnn()
encoder, decoder, vae_classifier = build_vae()

# Full pipeline: CNN → VAE Classifier
input_eeg = layers.Input(shape=(93, 32, 1))
cnn_features = cnn(input_eeg)
predictions = vae_classifier(cnn_features)
cnn_vae = models.Model(input_eeg, predictions)

# Compile
cnn_vae.compile(optimizer=optimizers.Adam(), 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

In [ ]:
# Reshape data for CNN (add channel dimension)
X_stft = X_stft[..., np.newaxis]  # Shape: (n_trials, 93, 32, 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_stft, y, test_size=0.1, random_state=42)

# Train CNN-VAE
history = cnn_vae.fit(X_train, y_train, 
                      epochs=100, 
                      batch_size=50, 
                      validation_data=(X_test, y_test))

# Evaluate
y_pred = (cnn_vae.predict(X_test) > 0.5).astype(int)
kappa = cohen_kappa_score(y_test, y_pred)
print(f"Kappa: {kappa:.3f}")